In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['stage1_sample_submission.csv', 'stage1_labels.csv', 'sample_images']


In [5]:
import pydicom
data_dir = '../input/sample_images/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('../input/stage1_labels.csv', index_col=0)
labels_df.head()
print(len(patients))

In [16]:
for patient in patients[:10]:
    label = labels_df.get_value(patient, 'cancer')
    path = os.path.join(data_dir, patient)
    
    # a couple great 1-liners from: https://www.kaggle.com/gzuidhof/data-science-bowl-2017/full-preprocessing-tutorial
    slices = [pydicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    print(len(slices),label, slices[0].pixel_array.shape)    

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


221 0 (512, 512)
126 0 (512, 512)
123 1 (512, 512)
244 0 (512, 512)
203 1 (512, 512)
136 1 (512, 512)
113 0 (512, 512)
435 1 (512, 512)
183 0 (512, 512)
180 0 (512, 512)
